### Лабораторная работа 2. Вариант №4
#### Торопкин Роман. 

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
np.set_printoptions(precision=3, suppress=True)

Исходные данные задачи. 

![Исходные данные](https://i.ibb.co/3r5v0Dq/Uralvn.jpg)

In [2]:
A_u = np.array([[1,2,4,8],
                 [3,5,1,0],
                [6,0,3,1]])
b_u = np.array([24,12,35])
sign = ['<=','<=','<=','<=']
c = np.array([4,2,5,8])

Формирование шапки строк и столбцов для симплексной таблицы

In [3]:
x_solution = [0 for i in range(0, len(c))]
sol_index = []
index_arr = []
columns_arr = []
sign_map_minus = {'>=','>'}
for i in range (0, len(b_u) + len(c)):
    columns_arr.append('x' + str(i + 1))
    if i < len(c):
        sol_index.append('x' + str(i + 1))
for i in range (len(b_u) + 1, len(b_u) * 2 + 1):
    index_arr.append('x' + str(i + 1))
index_arr.append('Z')

columns_arr.append('b')
sol_index.append('Z')

Шапка столбца

In [4]:
columns_arr

['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'b']

Шапка строк

In [5]:
index_arr

['x5', 'x6', 'x7', 'Z']

Матрица дополнительных переменных Xn+1..Xn+m

In [6]:
basis = np.eye(len(b_u), dtype='float64')
for i in range(0,len(b_u)):
    if sign[i] in sign_map_minus:
        basis[i][i] = -1
basis

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

#### Сгенерированная исходная симплексная таблица

In [7]:
b_u_T = []
for b in b_u:
    b_u_T.append([b])
b_u_T = np.array(b_u_T)
simplex_table = np.hstack([A_u, basis])
simplex_table = np.hstack([simplex_table, b_u_T])
Z_row = np.append(-c, np.zeros((1, len(b_u) +1)))
simplex_table = np.vstack([simplex_table, Z_row])
df = pd.DataFrame(data=simplex_table, index=index_arr, columns=columns_arr)
df

,x1,x2,x3,x4,x5,x6,x7,b
x5,1.0,2.0,4.0,8.0,1.0,0.0,0.0,24.0
x6,3.0,5.0,1.0,0.0,0.0,1.0,0.0,12.0
x7,6.0,0.0,3.0,1.0,0.0,0.0,1.0,35.0
Z,-4.0,-2.0,-5.0,-8.0,0.0,0.0,0.0,0.0


Функция определения разрешаюшего столбца

In [8]:
def find_permitting_column(row):
    mini = 0
    min_index = -1
    for i in range(0, len(row)):
        if row[i] < mini:
            min_index = i
            mini = row[i]
    return min_index

Функция определения разрешающей строки. Вычисление симплексных отношений

In [9]:
def find_permitting_row(table, p):
    mini = np.inf
    min_index = -1
    size = np.shape(table)
    for i in range(0, size[0]-1):
        if table[i][p] > 0:
            s = table[i][size[1] - 1] / table[i][p]
            if s < mini:
                mini = s
                min_index = i
    return min_index

Функция вычисления нового допустимого базисного решения
            

In [10]:
def calculate(table, p, q):
    size = np.shape(table)
    qp = table[q][p]
    for j in range(0, size[1] - 1):
        table[q][j] = table[q][j] / qp
    table[q][size[1] - 1] = table[q][size[1] - 1] / qp
    for i in range(0, size[0]):
        if i != q:
            ip = -table[i][p]
            for j in range(0, size[1] - 2):
                table[i][j] = table[i][j] + table[q][j] * ip
            table[i][size[1] - 1] = table[i][size[1] - 1] + table[q][size[1] - 1] * ip
    return table

Проверка признака неограниченности целевой функции

In [11]:
def check_solution(table, p):
    count = 0
    size = np.shape(table)
    for i in range(0, size[0]):
        if table[i][p] <= 0:
            count+=1
    if count == size[0]:
        return False
    else:
        return True


Симплекс-метод

In [12]:
iter_count = 0
last_index = np.shape(simplex_table)[0] - 1

# Проверка текущего допустимого базисного решения на оптимальность. 
# Сразу же определяем разрешающий столбец
permitting_column = find_permitting_column(simplex_table[last_index])

# Определение разрешающей строки
permitting_row = find_permitting_row(simplex_table, permitting_column)
print(f'{iter_count} итерация ')
print(df)
print(f'Резрешающая строка: {permitting_row + 1}, Резрешающий столбец {permitting_column + 1}')

# Проверка признака неогранниченности Z
isSolution = check_solution(simplex_table, permitting_column)
print(f'Существует решение - {isSolution}','', sep='\n')
index_arr[permitting_row] = columns_arr[permitting_column]

# Переход к новому базису
while permitting_column != -1 and isSolution:
    iter_count+=1
    print(f'{iter_count} итерация ')
    simplex_table = calculate(simplex_table, permitting_column, permitting_row)
    df = pd.DataFrame(data=simplex_table, index=index_arr, columns=columns_arr)
    print(df)
    permitting_column = find_permitting_column(simplex_table[last_index])
    permitting_row = find_permitting_row(simplex_table, permitting_column)
    print(f'Резрешающая строка: {permitting_row + 1}, Резрешающий столбец {permitting_column + 1}')
    isSolution = check_solution(simplex_table, permitting_column)
    print(f'Существует решение - {isSolution}','', sep='\n')
    index_arr[permitting_row] = columns_arr[permitting_column]

0 итерация 
     x1   x2   x3   x4   x5   x6   x7     b
x5  1.0  2.0  4.0  8.0  1.0  0.0  0.0  24.0
x6  3.0  5.0  1.0  0.0  0.0  1.0  0.0  12.0
x7  6.0  0.0  3.0  1.0  0.0  0.0  1.0  35.0
Z  -4.0 -2.0 -5.0 -8.0  0.0  0.0  0.0   0.0
Резрешающая строка: 1, Резрешающий столбец 4
Существует решение - True

1 итерация 
       x1    x2   x3   x4     x5   x6   x7     b
x4  0.125  0.25  0.5  1.0  0.125  0.0  0.0   3.0
x6  3.000  5.00  1.0  0.0  0.000  1.0  0.0  12.0
x7  5.875 -0.25  2.5  0.0 -0.125  0.0  1.0  32.0
Z  -3.000  0.00 -1.0  0.0  1.000  0.0  0.0  24.0
Резрешающая строка: 2, Резрешающий столбец 1
Существует решение - True

2 итерация 
     x1         x2        x3   x4     x5        x6   x7     b
x4  0.0   0.041667  0.458333  1.0  0.125 -0.041667  0.0   2.5
x1  1.0   1.666667  0.333333  0.0  0.000  0.333333  0.0   4.0
x7  0.0 -10.041667  0.541667  0.0 -0.125 -1.958333  1.0   8.5
Z   0.0   5.000000  0.000000  0.0  1.000  1.000000  0.0  36.0
Резрешающая строка: 1, Резрешающий столбец 0


Колличество итераций 

In [13]:
iter_count

2

Итоговое решение задачи

In [14]:
for i in range(0, len(c)):
    col = 'x' + str(i+1)
    if col in df.index.values:
        x_solution[i] = df['b'][col]
x_solution.append(df['b']['Z'])
pd.DataFrame(data=[x_solution], columns=sol_index)

,x1,x2,x3,x4,Z
0,4.0,0,0,2.5,36.0


Проверка решения встроенным симплекс - методом

In [15]:
inner_solution = linprog(-c, A_u, b_u, method='simplex')
print(f'Минимальное значение x: {inner_solution.x}\nОптимальное решение: {-inner_solution.fun}')

Минимальное значение x: [0. 0. 0. 0.]
Оптимальное решение: -0.0
